<a href="https://colab.research.google.com/github/Carocschall/CoderHouse/blob/main/DataScienceIII_CortezSchall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATA SCIENCE III: NLP Y DEEP LEARNING APLICADO A LA CIENCIA DE DATOS - Sentiment Analysis**



Alumna: Carolina Cortez Schall

# **Presentación del proyecto**

Abstracto con Motivación y Audiencia

Contexto Comercial y Analítico

Preguntas/Hipótesis a Resolver

Objetivo

# **Lectura de datos**

### **Librerias necesarias**

In [ ]:
!python -m spacy download es_core_news_sm
! pip install -U symspellpy
import nltk # importar natural language toolkit
nltk.download('punkt')
nltk.download('stopwords') # modulo para descargar stopwords en diferentes idiomas
nltk.download('wordnet')
from nltk.corpus import stopwords
import pandas as pd
import numpy  as np
import re
import string
import plotly
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
import time
import spacy
import es_core_news_sm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from wordcloud import WordCloud
import pickle
from symspellpy import SymSpell
import pkg_resources
from symspellpy import SymSpell, Verbosity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 39.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 13.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-2-37b7ca46bafc>:27: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


## **Lectura del corpus**

In [12]:
import kagglehub

# Download the latest version.
kagglehub.dataset_download('bricevergnou/spotify-recommendation')

# Download a specific version.
kagglehub.dataset_download('bricevergnou/spotify-recommendation/versions/1')

# Download a single file
kagglehub.dataset_download('bricevergnou/spotify-recommendation', path='data.csv')

# Download a dataset or file, even if previously downloaded to cache.
kagglehub.dataset_download('bricevergnou/spotify-recommendation', force_download=True)

100%|██████████| 29.5k/29.5k [00:00<00:00, 16.3MB/s]

Extracting files...


100%|██████████| 5.94k/5.94k [00:00<00:00, 5.76MB/s]

Extracting files...


Download already complete (14374 bytes).


100%|██████████| 29.5k/29.5k [00:00<00:00, 20.6MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/bricevergnou/spotify-recommendation/versions/2'

In [1]:
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 16.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import es_core_news_md
nlp = es_core_news_md.load()

# **Análisis inicial**

# **Análisis Exploratorio**

**Análisis de la Longitud de las Oraciones**

**Análisis de la Frecuencia de Palabras**

In [16]:
import numpy as np

# Assuming 'path' is a list of lists or a 2D numpy array containing strings:

# Create a dictionary to store word frequencies
frecuencia_total = {}

# Iterate through all strings in 'path'
for sublist in path:  # If path is a list of lists
    for word in sublist:
        frecuencia_total[word] = frecuencia_total.get(word, 0) + 1
        # If word is not in dictionary, initialize to 0 and add 1
        # If word is in dictionary, increment count by 1

# Convert the dictionary to a list of (word, frequency) tuples
frecuencia_total_list = list(frecuencia_total.items())

# Sort the list by frequency in descending order
frecuencia_total_list.sort(key=lambda item: item[1], reverse=True)

# Get the top 5 most frequent words and their frequencies
palabras_mas_frecuentes = [word for word, freq in frecuencia_total_list[:5]]
frecuencias_mas_frecuentes = [freq for word, freq in frecuencia_total_list[:5]]

# Print the results
print("Palabras más frecuentes:")
for palabra, frecuencia in zip(palabras_mas_frecuentes, frecuencias_mas_frecuentes):
    print(f"{palabra}: {frecuencia}")

Palabras más frecuentes:
/: 8
e: 5
s: 5
o: 4
a: 4


**Análisis de las Partes del Discurso (POS)**

**Análisis de la Distribución de Longitud de Palabras**

**Distribución de la Frecuencia de Palabras Únicas**

**Análisis de N-gramas**

**Análisis de la Diversidad Léxica**

**Visualización de Palabras con Word Cloud**

**Conclusiones del Analisis exploratorio**

# **Preprocesamiento del Texto**

Tokenización:

Tokenizar el texto en palabras, frases o párrafos según sea necesario.

Limpieza del Texto:

Convertir a minusculas, eliminar caracteres no deseados, stopwords, lematización, y stemming.

Análisis Léxico y Morfológico:

Identificación de partes del discurso (POS tagging) y análisis morfológico.

**Análisis Sintáctico**

Parsing:

Construir árboles sintácticos para las oraciones en el corpus.

Dependencia Sintáctica:

Análisis de dependencias para entender las relaciones gramaticales entre palabras.

**Análisis Semántico**

Coreferencia:

Resolver las referencias cruzadas en el texto para entender a qué se refieren los pronombres y otras expresiones.

Análisis de Coherencia:

Evaluar la coherencia y cohesión del discurso.


**Análisis Pragmático**

Análisis de Sentimientos:

Determinar el tono y las emociones expresadas en el texto.

Detección de Intenciones:

Identificar las intenciones detrás de las expresiones del texto.

**Codificación de texto a vectores**

Bag of word

Tf-IDF

Word Embendings

# **Feature Selection**

**Selección de variable objetivo y variables independientes**

**Clasificación de Texto**

X (Entrada): Texto del documento (puede ser una oración, párrafo, o artículo completo).

Y (Salida): Etiqueta de la categoría del texto (por ejemplo, "positivo" o "negativo" para análisis de sentimientos, o categorías como "deportes", "política", "tecnología" para clasificación de noticias).

**Análisis de Sentimientos**

X (Entrada): Texto del documento.

Y (Salida): Sentimiento asociado (por ejemplo, "positivo", "negativo" o "neutral").

**Reconocimiento de Entidades Nombradas (NER)**

X (Entrada): Texto del documento.

Y (Salida): Entidades reconocidas y sus categorías (por ejemplo, "PERSONA", "ORGANIZACIÓN", "LUGAR").

**Traducción Automática**

X (Entrada): Texto en el idioma de origen.

Y (Salida): Texto traducido al idioma de destino.

**Resumen de Texto**

X (Entrada): Texto completo del documento.

Y (Salida): Resumen del documento.

Generación de Texto **texto en negrita**

X (Entrada): Prompt o inicio de una frase o párrafo.

Y (Salida): Texto generado continuando el prompt.

**Modelos de Lenguaje (Language Modeling)**

X (Entrada): Una secuencia de palabras o caracteres.

Y (Salida): La siguiente palabra o carácter en la secuencia.

# **Modelos**

**Librerias necesarias para implementar los modelos**

**División de datos en conjuntos de entrenamiento y prueba**

**Predicción con conjunto de prueba**

**Evaluación del rendimiento del modelo**

**Conclusión sobre el modelado y las metricas**

# **Optimización de modelos**

**Conclusión sobre la optimización**

# **Conclusiones Finales**

# **Modelos utilizando Redes Neuronales**

**Definición del problema**

Determinar qué quieres lograr con la red neuronal (clasificación, regresión, predicción de series temporales, etc.).

Identificar las variables de entrada (features) y la salida esperada (labels o targets).

Según el problema, seleccionar el tipo adecuado: Perceptrón Multicapa (MLP), CNN, RNN, etc.

**Diseño de la red neuronal**

Estructura:

Seleccionar la cantidad de capas (profundidad) y neuronas por capa (ancho). Escoger la función de activación para cada capa (ReLU, Sigmoid, Softmax, etc.).

Conexiones:

Definir cómo se conectan las capas (densa, convolucional, recurrente, etc.). Pérdida y optimización:

Elegir una función de pérdida (Cross-Entropy, MSE, etc.) según el problema. Seleccionar un optimizador (SGD, Adam, etc.).

**Entrenamiento del modelo**

Inicialización: Iniciar pesos y sesgos de manera adecuada (aleatorio, Xavier, He, etc.).

Propagación hacia adelante (Forward pass): Calcular las predicciones. Cálculo de pérdida: Comparar las predicciones con las etiquetas esperadas.

Propagación hacia atrás (Backpropagation): Ajustar los pesos utilizando el gradiente descendente.

Iteraciones (Epochs): Repetir los pasos anteriores hasta alcanzar un buen desempeño o una condición de parada.

**Validación y ajuste**

**Evaluación del modelo**

**Visualización de resultados**

# **Conclusiones Finales**

# **Tipos de modelo de Redes Neuronales**

El modelo CNN es bueno para datos espaciales (como imágenes).

El modelo RNN es ideal para datos secuenciales o dependientes del tiempo.

**Modelo CNN Típico**

**Capa convolucional:**

Detecta patrones básicos en las imágenes, como bordes o texturas.

**MaxPooling:**

Reduce la dimensionalidad espacial para simplificar el modelo.

**Flatten:**

Convierte la salida bidimensional en un vector para pasar a las capas densas.

**Dropout:**

Previene el sobreajuste al apagar algunas neuronas aleatoriamente.

Capa densa (Activación Sigmoide o Softmax):

Genera probabilidades de pertenencia para cada clase.

**Modelo RNN Típico**

**Capa Embedding:**

Convierte palabras o categorías en vectores densos

**Capa RNN, LSTM o GRU:**

Captura dependencias temporales en las secuencias de datos.

**Dropout:**

Reduce el riesgo de sobreajuste.

**Capas densas:**
Procesan las características aprendidas para clasificar las secuencias.